In [1]:
from numpy import *
from scipy.sparse import *
from sklearn.preprocessing import *

In [2]:
#trackDuration=genfromtxt("tracks_final.csv",usecols=[0,2],skip_header=1,dtype=[("trackID",int32),("duration",int32)])

In [3]:
#trackDuration.sort(order="trackID")

In [4]:
#trackDuration=array(list(map(lambda x: [x[0],x[1]],trackDuration)))
trackDuration=load("fucking_ordered_track_duration.npy")

In [5]:
trackDuration

array([[    252,  140000],
       [    360,  194000],
       [    394,  250000],
       ..., 
       [5007354,  364000],
       [5014702,  156000],
       [5018274,      -1]], dtype=int32)

In [6]:
trackDuration.shape

(100000, 2)

In [7]:
#save("fucking_ordered_track_duration.npy",trackDuration)

In [8]:
mymask=where(trackDuration[:,1]==-1)[0]
A=array(ones(100000)).astype(int32)
A[mymask]=0# non c'è bisogno di fare multiply, controllare nel MAP
mymask.size,A.nonzero()[0].size

(2789, 97211)

In [8]:
testA=array([[1,100],[2,11],[3,18],[4,130]])
durationstest=testA[:,1]
k=0.6
l=[]
for s in range(4):
    value=durationstest[0]/durationstest[s]
    value=min(value,1/value)
    if(s==0 or value<k):
        l+=[0]
    else:
        l+=[value]
print(l)
firsttest=lil_matrix(l)
for i in range(1,4):
    l=[]
    for s in range(4):
        value=durationstest[i]/durationstest[s]
        value=min(value,1/value)
        if(s==i or value<k):
            l+=[0]
        else:
            l+=[value]
    print(l)
    firsttest=vstack((firsttest,lil_matrix(l)))
firsttest.data

[0, 0, 0, 0.76923076923076927]
[0, 0, 0.61111111111111116, 0]
[0, 0.61111111111111105, 0, 0]
[0.76923076923076916, 0, 0, 0]


array([ 0.76923077,  0.61111111,  0.61111111,  0.76923077])

In [9]:
durations=trackDuration[:,1]

In [ ]:
# non è lungo, è INFINITO
k=0.96 # viene densa circa al 7%
#qualche migliaio di canzoni per riga
l=[]
d=durations[0]
for s in range(100000):
    value=d/durations[s]
    value=min(value,1/value)
    if(s==0 or value<k):
        l+=[0]
    else:
        l+=[value]
#print(l)
j=1
firstRow=lil_matrix(l)
for i in range(1,100000):
    #break
    l=[]
    d=durations[i]
    for s in range(100000):
        value=d/durations[s]
        value=min(value,1/value)
        if(s==i or value<k):
            l+=[0]
        else:
            l+=[value]
    if(j%100==0):
        print(j)
    j+=1
    firstRow=vstack((firstRow,lil_matrix(l)))

In [14]:
d=durations[0]
valueArr=d/durations
valueArrInv=1/valueArr
values=array(list(map(lambda x:min(valueArr[x],valueArrInv[x]),range(100000))))
values[where(values<0.95)[0]]=0
values[0]=0
firstRow=csr_matrix(values)
firstRow

/usr/lib/python3/dist-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in true_divide
  


<1x100000 sparse matrix of type '<class 'numpy.float64'>'
	with 2292 stored elements in Compressed Sparse Row format>

In [ ]:
# MEMORY ERROR
for i in range(1,100000):
    d=durations[i]
    valueArr=d/durations
    valueArrInv=1/valueArr
    values=array(list(map(lambda x:min(valueArr[x],valueArrInv[x]),range(100000))))
    values[where(values<0.95)[0]]=0
    values[i]=0
    firstRow=vstack((firstRow,csr_matrix(values)))
    if (i%1000==0):
        print(i)